In [1]:
!pip install gnss_lib_py
import numpy as np
import gnss_lib_py as glp
import matplotlib.pyplot as plt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.3 MB/s eta 0:0

In [2]:
fix_data = glp.AndroidRawFixes("/content/gnss_log_2024_11_21_12_57_35.txt")

In [3]:
fix_types = np.unique(fix_data["fix_provider"]).tolist()
fix_types

['fused', 'gnss', 'network']

In [4]:
fixes = []
for provider in fix_types:
    fix_provider = fix_data.where("fix_provider",provider)
    fix_provider.rename({"lat_rx_deg":"lat_rx_" + provider + "_deg",
                         "lon_rx_deg":"lon_rx_" + provider + "_deg",
                         "alt_rx_m":"alt_rx_" + provider + "_m",
                         }, inplace=True)
    fixes.append(fix_provider)

In [5]:
raw_data = glp.AndroidRawGnss(input_path="/content/gnss_log_2024_11_21_12_57_35.txt",
                              filter_measurements=True,
                              measurement_filters={"sv_time_uncertainty" : 500.},
                              verbose=True)

sv_time_uncertainty removed 102


In [6]:
if isinstance(raw_data, dict):
    df = pd.DataFrame(raw_data)
else:
    df = raw_data

# Save the DataFrame to a CSV file
csv_file_path = "raw_data.csv"  # Define the output file name
df.to_csv(csv_file_path, index=False)

print(f"Data has been saved to {csv_file_path}")


Data has been saved to raw_data.csv


In [7]:
full_states = glp.add_sv_states(raw_data, source="precise", verbose=False)

In [ ]:
if isinstance(full_states, dict):
    df = pd.DataFrame(full_states)
else:
    df = full_states

# Save the DataFrame to a CSV file
csv_file_path = "full_states.csv"  # Define the output file name
df.to_csv(csv_file_path, index=False)

print(f"Data has been saved to {csv_file_path}")


Data has been saved to full_states.csv


In [9]:
derived_states = full_states.copy()
derived_states["corr_pr_m"] = derived_states["raw_pr_m"] \
                            + derived_states['b_sv_m']
# remove all data with raw_pr_m > 1e8
derived_states = derived_states.where("raw_pr_m", 3e7, "leq")

In [10]:
if isinstance(derived_states, dict):
    df = pd.DataFrame(derived_states)
else:
    df = derived_states

# Save the DataFrame to a CSV file
csv_file_path = "derived_states.csv"  # Define the output file name
df.to_csv(csv_file_path, index=False)

print(f"Data has been saved to {csv_file_path}")

Data has been saved to derived_states.csv
